In [2]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras

    100% |████████████████████████████████| 184kB 518kB/s 
  Running setup.py bdist_wheel for rpy2 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/23/9e/ee/0e5f6a00aafef9935d40ebf7657278220139f0101321e30d07
Successfully built rpy2


In [1]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam

Using TensorFlow backend.


## 1. DATA PREPARATION

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
known   = pd.read_csv(path + "data/train.csv")
unknown = pd.read_csv(path + "data/test.csv")

In [3]:
# keep only Flow songs in the data
# this proves to predict better, but some information is lost
known = known.query("listen_type == 1")
known.shape

(2319611, 15)

In [5]:
# Prepare the data
# Create a placeholder for the IDs new in the test data
newUsers   = list(unknown.user_id[~unknown.user_id.isin(known.user_id)])       + list(known.user_id.value_counts().keys()[known.user_id.value_counts()     == 1])
newSongs   = list(unknown.media_id[~unknown.media_id.isin(known.media_id)])    + list(known.media_id.value_counts().keys()[known.media_id.value_counts()   == 1])[-1000:]
newArtists = list(unknown.artist_id[~unknown.artist_id.isin(known.artist_id)]) + list(known.artist_id.value_counts().keys()[known.artist_id.value_counts() == 1])[-1000:]
newGenres  = list(unknown.genre_id[~unknown.genre_id.isin(known.genre_id)])    + list(known.genre_id.value_counts().keys()[known.genre_id.value_counts()   == 1])[-1000:]
newContext = list(unknown.context_type[~unknown.context_type.isin(known.context_type)]) + list(known.context_type.value_counts().keys()[known.context_type.value_counts() == 1])[-1000:]

# In particular, assign IDs from 0 to N for users and songs
# Use enumerate() to create a list of the new and original IDs
users   = list(enumerate([i for i in known.user_id.unique()   if i not in newUsers]))
songs   = list(enumerate([i for i in known.media_id.unique()  if i not in newSongs]))
artists = list(enumerate([i for i in known.artist_id.unique() if i not in newArtists]))
genres  = list(enumerate([i for i in known.genre_id.unique()  if i not in newGenres]))
context = list(enumerate([i for i in known.context_type.unique() if i not in newContext]))

# Create a dictionary with old IDs to new IDs
userid2idx    = {o:i for i,o in users}
songid2idx    = {o:i for i,o in songs}
artistid2idx  = {o:i for i,o in artists}
genreid2idx   = {o:i for i,o in genres}
contextid2idx = {o:i for i,o in context}

# Update with new/rare entries
userid2idx.update({o:(max(userid2idx.values())+1)       for o in newUsers})
songid2idx.update({o:(max(songid2idx.values())+1)       for o in newSongs})
artistid2idx.update({o:(max(artistid2idx.values())+1)   for o in newArtists})
genreid2idx.update({o:(max(genreid2idx.values())+1)     for o in newGenres})
contextid2idx.update({o:(max(contextid2idx.values())+1) for o in newContext})

# Create id variable with the new IDs (known)
known['userIdx']    = known.user_id.apply(lambda x:       userid2idx[x])
known['songIdx']    = known.media_id.apply(lambda x:      songid2idx[x])
known['artistIdx']  = known.artist_id.apply(lambda x:     artistid2idx[x])
known['genreIdx']   = known.genre_id.apply(lambda x:      genreid2idx[x])
known['contextIdx'] = known.context_type.apply(lambda x:  contextid2idx[x])

# Create id variable with the new IDs (unknown)
unknown['userIdx']    = unknown.user_id.apply(lambda x:      userid2idx[x])
unknown['songIdx']    = unknown.media_id.apply(lambda x:     songid2idx[x])
unknown['artistIdx']  = unknown.artist_id.apply(lambda x:    artistid2idx[x])
unknown['genreIdx']   = unknown.genre_id.apply(lambda x:     genreid2idx[x])
unknown['contextIdx'] = unknown.context_type.apply(lambda x: contextid2idx[x])

In [6]:
# partition train/test data: last 3 songs per user go to validation (stage 1)
# model predicts better if trained on full known sample without validation (stage 2)
ts = known.groupby(["userIdx"]).tail(3)
tr = known.groupby(["userIdx"], group_keys=False).apply(lambda x: x[:-3])

# move songs that appear only in ts to tr
strayObs = ts.songIdx.isin(tr.songIdx) & ts.userIdx.isin(tr.userIdx) & ts.artistIdx.isin(tr.artistIdx) & ts.genreIdx.isin(tr.genreIdx) & ts.contextIdx.isin(tr.contextIdx)
tr = tr.append(ts[~strayObs])
ts = ts[strayObs]

## 2. MODELING

### 2.1. INITIALIZING

In [7]:
# create an input layer with one row of IDs
user_in    = Input(shape = (1,), dtype='int64',   name = "user_in")
song_in    = Input(shape = (1,), dtype='int64',   name = "song_in")
artist_in  = Input(shape = (1,), dtype='int64',   name = "artist_in")
genre_in   = Input(shape = (1,), dtype='int64',   name = "genre_in")
context_in = Input(shape = (1,), dtype='int64',   name = "context_in")
num_in     = Input(shape = (1,), dtype = 'float32', name = "num_in")

# Reshaping numeric features
n = Reshape((1,1))(num_in)

# Create an embedding assigning k latent factors to each ID
# These will be optimized
# A regulariztaion is added to avoid very large weights
n_users   = tr.userIdx.nunique()
n_songs   = tr.songIdx.nunique()
n_artists = tr.artistIdx.nunique()
n_genres  = tr.genreIdx.nunique()
n_context = tr.contextIdx.nunique()

# Embeddings creation
u = Embedding(n_users,   50, input_length=1, embeddings_regularizer=l2(1e-5))(user_in)
s = Embedding(n_songs,   50, input_length=1, embeddings_regularizer=l2(1e-5))(song_in)
a = Embedding(n_artists, 50, input_length=1, embeddings_regularizer=l2(1e-5))(artist_in)
g = Embedding(n_genres,  50, input_length=1, embeddings_regularizer=l2(1e-5))(genre_in)
c = Embedding(n_context, 50, input_length=1, embeddings_regularizer=l2(1e-5))(context_in)

# Specify what to do with the layers
#x = concatenate([u, s, a, g, c, n]) # with one numeric feature
x = concatenate([u, s, a, g, c])     # without numeric features
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)

# Then we specify the model that we want to use
#model = Model([user_in, song_in, artist_in, genre_in, context_in, num_in], x) # with one numeric feature
model = Model([user_in, song_in, artist_in, genre_in, context_in], x)          # without numeric features
model.compile(Adam(0.001), loss = "binary_crossentropy", metrics = ['accuracy'])

### 2.2. FIRST STAGE

In [9]:
# run the estimations on training data
model.fit([tr.userIdx, tr.songIdx, tr.artistIdx, tr.genreIdx, tr.contextIdx], tr.is_listened, 
validation_data = ([ts.userIdx, ts.songIdx, ts.artistIdx, ts.genreIdx, ts.contextIdx], ts.is_listened),
batch_size = int(len(tr)/100), epochs = 10)

Train on 2281332 samples, validate on 38279 samples
Epoch 1/10
2281332/2281332 [==============================] - 74s - loss: 0.5962 - acc: 0.6996 - val_loss: 0.6802 - val_acc: 0.5668
Epoch 2/10
2281332/2281332 [==============================] - 110s - loss: 0.5010 - acc: 0.7672 - val_loss: 0.6591 - val_acc: 0.5947
Epoch 3/10
2281332/2281332 [==============================] - 114s - loss: 0.4863 - acc: 0.7744 - val_loss: 0.6318 - val_acc: 0.6604
Epoch 4/10
2281332/2281332 [==============================] - 114s - loss: 0.4754 - acc: 0.7799 - val_loss: 0.6034 - val_acc: 0.7001
Epoch 5/10
2281332/2281332 [==============================] - 119s - loss: 0.4672 - acc: 0.7844 - val_loss: 0.5714 - val_acc: 0.7257
Epoch 6/10
2281332/2281332 [==============================] - 68s - loss: 0.4606 - acc: 0.7883 - val_loss: 0.5518 - val_acc: 0.7361
Epoch 7/10
2281332/2281332 [==============================] - 67s - loss: 0.4545 - acc: 0.7920 - val_loss: 0.5503 - val_acc: 0.7413
Epoch 8/10
2281332/2

### 2.3. SECOND STAGE

In [61]:
# run the estimations on full known data
model.fit([known.userIdx, known.songIdx, known.artistIdx, known.genreIdx, known.contextIdx], known.is_listened,
batch_size = int(known.shape[0]/100), epochs = 10)

Epoch 1/10
2319611/2319611 [==============================] - 113s - loss: 0.4335 - acc: 0.8019   
Epoch 2/10
2319611/2319611 [==============================] - 113s - loss: 0.4286 - acc: 0.8044   
Epoch 3/10
2319611/2319611 [==============================] - 112s - loss: 0.4245 - acc: 0.8064   
Epoch 4/10
2319611/2319611 [==============================] - 112s - loss: 0.4213 - acc: 0.8083   
Epoch 5/10
2319611/2319611 [==============================] - 110s - loss: 0.4184 - acc: 0.8095   
Epoch 6/10
2319611/2319611 [==============================] - 113s - loss: 0.4152 - acc: 0.8113   
Epoch 7/10
2319611/2319611 [==============================] - 112s - loss: 0.4125 - acc: 0.8126   
Epoch 8/10
2319611/2319611 [==============================] - 113s - loss: 0.4106 - acc: 0.8139   
Epoch 9/10
2319611/2319611 [==============================] - 115s - loss: 0.4082 - acc: 0.8152   
Epoch 10/10
2319611/2319611 [==============================] - 112s - loss: 0.4058 - acc: 0.8163   


# 3. FORECASTING

In [64]:
# predict unknown data
pred = pd.DataFrame()
pred["sample_id"] = unknown.sample_id
pred["is_listened"] = model.predict([unknown.userIdx, unknown.songIdx, unknown.artistIdx, unknown.genreIdx, unknown.contextIdx])
pred.head(5)

,sample_id,is_listened
0,0,0.999775
1,1,0.649338
2,2,0.748015
3,3,0.525618
4,4,0.983196


In [65]:
# loading naive submission
naive = pd.read_csv(path + "submissions/naive_ratio_user.csv")

In [66]:
# combining predictions
pred_mean = pred
pred_mean["is_listened"] = 0.6*pred["is_listened"] + 0.4*naive["is_listened"]
pred_mean.head(5)

,sample_id,is_listened
0,0,0.999865
1,1,0.699207
2,2,0.800616
3,3,0.557314
4,4,0.946639


In [67]:
# saving submissions
pred.to_csv(path      + "submissions/deep_128_64_flow_cont.csv", index = False)
pred_mean.to_csv(path + "submissions/deep_128_64_flow_cont_plus_ratio_user_0_6.csv", index = False)